In [1]:
import pandas as pd
import numpy as np
from preprocess import k_user_interactions, k_core
from utils import load_raw_df, df_to_sparse, gini

In [2]:
def get_stat_dict(rating_matrix):
    NUM_USERS, NUM_ITEMS = rating_matrix.shape
    NUM_RATINGS = rating_matrix.nnz
    NUM_RATINGS_PER_USER = NUM_RATINGS / NUM_USERS

    DENSITY = NUM_RATINGS / (NUM_USERS * NUM_ITEMS)
    SPARSITY = 1 - DENSITY
    SHAPE = NUM_USERS / NUM_ITEMS
    
    user_popularity = rating_matrix.sum(1).A.reshape(-1)
    item_popularity = rating_matrix.sum(0).A.reshape(-1)

    sorted_user_popularity = np.sort(user_popularity)
    sorted_item_popularity = np.sort(item_popularity)

    GINI_USER = gini(sorted_user_popularity)
    GINI_ITEM = gini(sorted_item_popularity)

    CONCENTRATION = sum(sorted_item_popularity[-int(len(item_popularity) * 0.05):]) / NUM_RATINGS

    ret = {
        '# Users': NUM_USERS,
        '# Items': NUM_ITEMS,
        '# Ratings': NUM_RATINGS,
        '# Ratings per user': NUM_RATINGS_PER_USER,
        'Sparsity': SPARSITY,
        'Shape': SHAPE,
        'Gini User': GINI_USER,
        'Gini Item': GINI_ITEM,
        'Concen.': CONCENTRATION
    }
    return ret


In [3]:
data_to_file = {
    'ml-1m': 'data/ml-1m/ratings.dat',
    'ml-10m': 'data/ml-10m/ratings.dat',
    'ml-20m': 'data/ml-20m/ratings.csv',
    'ml-25m': 'data/ml-25m/ratings.csv',
    'amusic': 'data/amusic/Digital_Music_5.json',
    'acd': 'data/acd/CDs_and_Vinyl_5.json',
    'acna': 'data/acna/Cell_Phones_and_Accessories_5.json',
    'agames': 'data/agames/Video_Games_5.json',
    'ciao': 'data/ciao/ratings.dat',
    'epinions': 'data/epinions/ratings_data.txt',
    'yelp2015': 'data/yelp2015/yelp.rating',
    'yelp2018': 'data/yelp2018/yelp.csv',
    'gowalla': 'data/gowalla/loc-gowalla_totalCheckins.txt',
    'citeulike': 'data/citeulike/users.dat',
    'pinterest': 'data/pinterest/pinterest.csv'
}
data_to_sep = {
    'ml-1m': '::',
    'ml-10m': '::',
    'ml-20m': ',',
    'ml-25m': ',',
    'amusic': ',',
    'acd': ',',
    'acna': ',',
    'agames': ',',
    'ciao': '\t',
    'epinions': ' ',
    'yelp2015': '\t',
    'yelp2018': ',',
    'gowalla': '\t',
    'citeulike': ' ',
    'pinterest': ','
}


In [4]:
dataset_list = list(data_to_file.keys())

In [5]:
raw_df_dict = {d: None for d in data_to_file}

# Original

In [6]:
%%time
stat_df = pd.DataFrame(
    columns=['dataset', '# Users', '# Items', '# Ratings', '# Ratings per user', 'Sparsity', 'Shape', 'Gini User', 'Gini Item', 'Concen.'])
for dataset in dataset_list:
    print(dataset)
    if raw_df_dict[dataset] is None:
        raw_df = load_raw_df(dataset, data_to_file, data_to_sep)
        raw_df_dict[dataset] = raw_df
    else:
        raw_df = raw_df_dict[dataset]
    
    raw_df = k_user_interactions(raw_df, 1)
    
    num_unique_users = len(pd.unique(raw_df.user))
    num_unique_items = len(pd.unique(raw_df.item))
    
    shape = (num_unique_users, num_unique_items)
    rating_matrix = df_to_sparse(raw_df, shape)

    stat_dict = get_stat_dict(rating_matrix)
    stat_dict['dataset'] = dataset
    stat_df = stat_df.append(stat_dict, ignore_index=True)

ml-1m
initial user, item: 6040 3706
after filter : 6040 3706
ml-10m
initial user, item: 69878 10677
after filter : 69878 10677
ml-20m
initial user, item: 138493 26744
after filter : 138493 26744
ml-25m
initial user, item: 162541 59047
after filter : 162541 59047
100%|██████████| 169781/169781 [00:01<00:00, 105510.75it/s]
initial user, item: 16566 11797
after filter : 16566 11797
acd
100%|██████████| 1443755/1443755 [00:18<00:00, 77517.94it/s]
initial user, item: 112395 73713
after filter : 112395 73713
acna
100%|██████████| 1128437/1128437 [00:10<00:00, 107448.46it/s]
initial user, item: 157212 48186
after filter : 157212 48186
agames
100%|██████████| 497577/497577 [00:05<00:00, 89380.10it/s]
initial user, item: 55223 17408
after filter : 55223 17408
ciao
initial user, item: 6762 16610
after filter : 6762 16610
epinions
initial user, item: 40163 139738
after filter : 40163 139738
yelp2015
initial user, item: 25677 25815
after filter : 25677 25815
yelp2018
initial user, item: 45919 4553

In [7]:
stat_df

,dataset,# Users,# Items,# Ratings,# Ratings per user,Sparsity,Shape,Gini User,Gini Item,Concen.
0,ml-1m,6040,3706,1000209,165.597517,0.955316,1.629790,0.528624,0.633562,0.282754
1,ml-10m,69878,10677,10000054,143.107330,0.986597,6.544722,0.570700,0.805249,0.516455
2,ml-20m,138493,26744,20000263,144.413530,0.994600,5.178470,0.580701,0.902942,0.714144
3,ml-25m,162541,59047,25000095,153.807932,0.997395,2.752739,0.589469,0.941897,0.844548
4,amusic,16566,11797,145292,8.770494,0.999257,1.404255,0.375600,0.431506,0.247570
5,acd,112395,73713,1402148,12.475181,0.999831,1.524765,0.465575,0.543188,0.335323
6,acna,157212,48186,1120771,7.129042,0.999852,3.262607,0.209092,0.587514,0.378678
7,agames,55223,17408,473427,8.573004,0.999508,3.172277,0.323024,0.584914,0.348421
8,ciao,6762,16610,146997,21.738687,0.998691,0.407104,0.551811,0.530903,0.312775
9,epinions,40163,139738,664823,16.553121,0.999882,0.287416,0.676261,0.693562,0.533856


# 10 user interactions

In [8]:
%%time
stat_df = pd.DataFrame(
    columns=['dataset', '# Users', '# Items', '# Ratings', '# Ratings per user', 'Sparsity', 'Shape', 'Gini User', 'Gini Item', 'Concen.'])
for dataset in dataset_list:
    print(dataset)
    if raw_df_dict[dataset] is None:
        raw_df = load_raw_df(dataset, data_to_file, data_to_sep)
        raw_df_dict[dataset] = raw_df
    else:
        raw_df = raw_df_dict[dataset]
    
    raw_df = k_user_interactions(raw_df, 10)
    
    num_unique_users = len(pd.unique(raw_df.user))
    num_unique_items = len(pd.unique(raw_df.item))
    
    shape = (num_unique_users, num_unique_items)
    rating_matrix = df_to_sparse(raw_df, shape)

    stat_dict = get_stat_dict(rating_matrix)
    stat_dict['dataset'] = dataset
    stat_df = stat_df.append(stat_dict, ignore_index=True)

ml-1m
initial user, item: 6040 3706
after filter : 6040 3706
ml-10m
initial user, item: 69878 10677
after filter : 69878 10677
ml-20m
initial user, item: 138493 26744
after filter : 138493 26744
ml-25m
initial user, item: 162541 59047
after filter : 162541 59047
amusic
initial user, item: 16566 11797
after filter : 3951 11483
acd
initial user, item: 112395 73713
after filter : 36487 73493
acna
initial user, item: 157212 48186
after filter : 21361 44416
agames
initial user, item: 55223 17408
after filter : 11986 17100
ciao
initial user, item: 6762 16610
after filter : 3829 15768
epinions
initial user, item: 40163 139738
after filter : 15786 132964
yelp2015
initial user, item: 25677 25815
after filter : 24930 25799
yelp2018
initial user, item: 45919 45538
after filter : 45842 45538
gowalla
initial user, item: 107092 1280969
after filter : 68709 1247158
citeulike
initial user, item: 5551 16980
after filter : 5551 16980
pinterest
initial user, item: 55187 9916
after filter : 55187 9916
CPU

In [9]:
stat_df

,dataset,# Users,# Items,# Ratings,# Ratings per user,Sparsity,Shape,Gini User,Gini Item,Concen.
0,ml-1m,6040,3706,1000209,165.597517,0.955316,1.629790,0.528624,0.633562,0.282754
1,ml-10m,69878,10677,10000054,143.107330,0.986597,6.544722,0.570700,0.805249,0.516455
2,ml-20m,138493,26744,20000263,144.413530,0.994600,5.178470,0.580701,0.902942,0.714144
3,ml-25m,162541,59047,25000095,153.807932,0.997395,2.752739,0.589469,0.941897,0.844548
4,amusic,3951,11483,75044,18.993672,0.998346,0.344074,0.306667,0.450453,0.240486
5,acd,36487,73493,933651,25.588593,0.999652,0.496469,0.444807,0.545487,0.329630
6,acna,21361,44416,301976,14.136791,0.999682,0.480930,0.191547,0.580696,0.347895
7,agames,11986,17100,215645,17.991407,0.998948,0.700936,0.299394,0.583335,0.331930
8,ciao,3829,15768,129798,33.898668,0.997850,0.242834,0.451588,0.526729,0.311900
9,epinions,15786,132964,580752,36.789054,0.999723,0.118724,0.481602,0.676767,0.519681


# 20 user interactions

In [10]:
%%time
stat_df = pd.DataFrame(
    columns=['dataset', '# Users', '# Items', '# Ratings', '# Ratings per user', 'Sparsity', 'Shape', 'Gini User', 'Gini Item', 'Concen.'])
for dataset in dataset_list:
    print(dataset)
    if raw_df_dict[dataset] is None:
        raw_df = load_raw_df(dataset, data_to_file, data_to_sep)
        raw_df_dict[dataset] = raw_df
    else:
        raw_df = raw_df_dict[dataset]
    
    raw_df = k_user_interactions(raw_df, 20)
    
    num_unique_users = len(pd.unique(raw_df.user))
    num_unique_items = len(pd.unique(raw_df.item))
    
    shape = (num_unique_users, num_unique_items)
    rating_matrix = df_to_sparse(raw_df, shape)

    stat_dict = get_stat_dict(rating_matrix)
    stat_dict['dataset'] = dataset
    stat_df = stat_df.append(stat_dict, ignore_index=True)

ml-1m
initial user, item: 6040 3706
after filter : 6040 3706
ml-10m
initial user, item: 69878 10677
after filter : 69878 10677
ml-20m
initial user, item: 138493 26744
after filter : 138493 26744
ml-25m
initial user, item: 162541 59047
after filter : 162541 59047
amusic
initial user, item: 16566 11797
after filter : 1074 10116
acd
initial user, item: 112395 73713
after filter : 12284 71838
acna
initial user, item: 157212 48186
after filter : 2302 25285
agames
initial user, item: 55223 17408
after filter : 2734 15189
ciao
initial user, item: 6762 16610
after filter : 2075 14887
epinions
initial user, item: 40163 139738
after filter : 8693 123330
yelp2015
initial user, item: 25677 25815
after filter : 9788 25373
yelp2018
initial user, item: 45919 45538
after filter : 17137 45447
gowalla
initial user, item: 107092 1280969
after filter : 47752 1183848
citeulike
initial user, item: 5551 16980
after filter : 3097 16792
pinterest
initial user, item: 55187 9916
after filter : 52190 9909
CPU tim

In [11]:
stat_df

,dataset,# Users,# Items,# Ratings,# Ratings per user,Sparsity,Shape,Gini User,Gini Item,Concen.
0,ml-1m,6040,3706,1000209,165.597517,0.955316,1.629790,0.528624,0.633562,0.282754
1,ml-10m,69878,10677,10000054,143.107330,0.986597,6.544722,0.570700,0.805249,0.516455
2,ml-20m,138493,26744,20000263,144.413530,0.994600,5.178470,0.580701,0.902942,0.714144
3,ml-25m,162541,59047,25000095,153.807932,0.997395,2.752739,0.589469,0.941897,0.844548
4,amusic,1074,10116,37526,34.940410,0.996546,0.106168,0.267339,0.446593,0.229787
5,acd,12284,71838,616845,50.215321,0.999301,0.170996,0.426236,0.547223,0.323405
6,acna,2302,25285,66772,29.006082,0.998853,0.091042,0.197624,0.459295,0.258477
7,agames,2734,15189,97227,35.562180,0.997659,0.179999,0.282589,0.561123,0.306880
8,ciao,2075,14887,105827,51.000964,0.996574,0.139383,0.381354,0.522431,0.308324
9,epinions,8693,123330,482849,55.544576,0.999550,0.070486,0.419244,0.652472,0.498334


# 10 core

In [12]:
%%time
stat_df = pd.DataFrame(
    columns=['dataset', '# Users', '# Items', '# Ratings', '# Ratings per user', 'Sparsity', 'Shape', 'Gini User', 'Gini Item', 'Concen.'])
for dataset in dataset_list:
    print(dataset)
    if raw_df_dict[dataset] is None:
        raw_df = load_raw_df(dataset, data_to_file, data_to_sep)
        raw_df_dict[dataset] = raw_df
    else:
        raw_df = raw_df_dict[dataset]
    
    raw_df = k_core(raw_df, 10)
    
    num_unique_users = len(pd.unique(raw_df.user))
    num_unique_items = len(pd.unique(raw_df.item))
    if num_unique_users == 0 or num_unique_items == 0:
        print('skip', dataset)
        continue
    
    shape = (num_unique_users, num_unique_items)
    rating_matrix = df_to_sparse(raw_df, shape)

    stat_dict = get_stat_dict(rating_matrix)
    stat_dict['dataset'] = dataset
    stat_df = stat_df.append(stat_dict, ignore_index=True)

ml-1m
initial user, item: 6040 3706
cycle 1 U 6040 -> 6040
cycle 1 I 3706 -> 3260
False True
cycle 2 U 6040 -> 6040
cycle 2 I 3260 -> 3260
False False
ml-10m
initial user, item: 69878 10677
cycle 1 U 69878 -> 69878
cycle 1 I 10677 -> 9708
False True
cycle 2 U 69878 -> 69878
cycle 2 I 9708 -> 9708
False False
ml-20m
initial user, item: 138493 26744
cycle 1 U 138493 -> 138493
cycle 1 I 26744 -> 15451
False True
cycle 2 U 138493 -> 138493
cycle 2 I 15451 -> 15451
False False
ml-25m
initial user, item: 162541 59047
cycle 1 U 162541 -> 162541
cycle 1 I 59047 -> 24330
False True
cycle 2 U 162541 -> 162539
cycle 2 I 24330 -> 24330
True False
cycle 3 U 162539 -> 162539
cycle 3 I 24330 -> 24330
False False
amusic
initial user, item: 16566 11797
cycle 1 U 16566 -> 3951
cycle 1 I 11797 -> 1935
True True
cycle 2 U 3951 -> 1190
cycle 2 I 1935 -> 750
True True
cycle 3 U 1190 -> 554
cycle 3 I 750 -> 365
True True
cycle 4 U 554 -> 309
cycle 4 I 365 -> 225
True True
cycle 5 U 309 -> 186
cycle 5 I 225 -

In [13]:
stat_df

,dataset,# Users,# Items,# Ratings,# Ratings per user,Sparsity,Shape,Gini User,Gini Item,Concen.
0,ml-1m,6040,3260,998539,165.321026,0.949288,1.852761,0.528514,0.586218,0.259408
1,ml-10m,69878,9708,9995471,143.041744,0.985266,7.197981,0.570571,0.786661,0.494494
2,ml-20m,138493,15451,19964833,144.157705,0.990670,8.963368,0.580155,0.836033,0.578370
3,ml-25m,162539,24330,24890566,153.135961,0.993706,6.680600,0.588092,0.871051,0.655785
4,acd,21450,18398,527503,24.592214,0.998663,1.165888,0.428618,0.427359,0.235085
5,agames,5942,3793,103778,17.465163,0.995395,1.566570,0.285697,0.402838,0.215768
6,ciao,2136,2597,59884,28.035581,0.989205,0.822487,0.398235,0.406278,0.285519
7,epinions,10706,8945,300303,28.049972,0.996864,1.196870,0.403722,0.494997,0.305511
8,yelp2015,22087,14873,602517,27.279259,0.998166,1.485040,0.441275,0.512086,0.293510
9,yelp2018,39055,25033,988768,25.317322,0.998989,1.560141,0.416771,0.506528,0.288847
